In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import os

In [2]:
'''
Loads Image data, resizes and normalizes them and returns as numpy arrays
params:
    base_path - path to the folder
    num_files - number of files to be loaded
    categories - names of sub-folders
returns:
    image_data and category_id as numpy arrays
'''
def readTextFromFiles(base_path, num_files, categories):
    c = len(categories)
    num_cat = int(num_files / c)
    X = []
    Y = []
    
    for (cat_num, category) in enumerate(categories):    
        i = 0
        file_path = base_path + category + '/'
        files = os.listdir(file_path)
        for file in files:
            if i >= num_cat:
                break
            img = image.load_img(file_path +file, target_size=(512, 512, 3), color_mode = 'rgb')
            X.append(image.img_to_array(img) / 255)
            Y.append(cat_num)
            i+=1
        
    return np.array(X), np.array(Y)

In [3]:
# Loads dataset of images of Rice leaves affected by 3 diseases
categories = ('blast', 'blight', 'tungro')
X, Y = readTextFromFiles('/kaggle/input/leaf-rice-disease-indonesia/', 240, categories)
print(X.shape)
print(Y.shape)

(240, 512, 512, 3)
(240,)


In [4]:
'''
Converts category indices to one-hot vectors
params:
    Y - arrays of categories
    num_cat - number of categories
returns:
    one hot representation of Y
'''
def convert_categories_to_onehot(Y, num_cat):
    Y_one_hot = np.eye(num_cat)[np.squeeze(Y).astype(int)]
    return Y_one_hot

In [5]:
# Converts category indices 0,1,2 into one hot vectors
Y_oh = convert_categories_to_onehot(Y, len(categories))
print(Y_oh.shape)

(240, 3)


In [6]:
# Splits the dataset into training set, validation set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_oh, test_size=0.2)
print(X_train.shape)
print(Y_train.shape)
val = int(X_test.shape[0]/2)
X_val = X_test[0:val,:,:,:]
Y_val = Y_test[0:val,:]
X_test = X_test[val:,:,:,:]
Y_test = Y_test[val:,:]
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

(192, 512, 512, 3)
(192, 3)
(24, 512, 512, 3)
(24, 3)
(24, 512, 512, 3)
(24, 3)


In [21]:
# Builds a Convolutional Neural Network Model
def get_model():
    model = tf.keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=(512,512,3)))
    
    # Data Augmentation to reduce overfitting to the training set
    model.add(keras.layers.experimental.preprocessing.RandomFlip('horizontal'))
    model.add(keras.layers.experimental.preprocessing.RandomRotation(0.1))
    
    # Convolutional Layer 1
    model.add(keras.layers.Conv2D(32, kernel_size=(3,3), strides=(2,2)))
    # Batch Normalization to speed up learning
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.ReLU())
    
    # Convolutional Layer 2
    model.add(keras.layers.Conv2D(64, kernel_size=(3,3), strides=(2,2)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.ReLU())
    
    # Max Pooling Layer 1
    model.add(keras.layers.MaxPooling2D(pool_size=(3,3)))
    # Dropout layer 1 for regularization (to reduce overfitting)
    model.add(keras.layers.Dropout(0.15))
    
    # Convolutional Layer 3
    model.add(keras.layers.Conv2D(128, kernel_size=(3,3), strides=(2,2)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.ReLU())
    
    # Max Pooling Layer 2
    model.add(keras.layers.MaxPooling2D(pool_size=(3,3)))
    # Dropout Layer 2
    model.add(keras.layers.Dropout(0.15))
    
    # Fully connected layers
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dense(3, activation='softmax'))
    
    return model

In [23]:
model = get_model()

# Compiles the model with Adam Optimizer
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0003), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_3 (RandomFlip)   (None, 512, 512, 3)       0         
_________________________________________________________________
random_rotation_3 (RandomRot (None, 512, 512, 3)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 255, 255, 32)      896       
_________________________________________________________________
batch_normalization_12 (Batc (None, 255, 255, 32)      128       
_________________________________________________________________
re_lu_12 (ReLU)              (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 127, 127, 64)      18496     
_________________________________________________________________
batch_normalization_13 (Batc (None, 127, 127, 64)     

In [25]:
# Training : Fits the CNN model to the training dataset using Mini-batch gradient descent
model.fit(X_train, Y_train, epochs=20, validation_data = (X_val, Y_val), batch_size=8)

Epoch 1/20
24/24 [==============================] - 18s 730ms/step - loss: 2.8939 - accuracy: 0.4726 - val_loss: 1.0655 - val_accuracy: 0.4167
Epoch 2/20
24/24 [==============================] - 17s 709ms/step - loss: 0.8938 - accuracy: 0.5803 - val_loss: 1.0580 - val_accuracy: 0.4167
Epoch 3/20
24/24 [==============================] - 17s 692ms/step - loss: 0.5076 - accuracy: 0.8077 - val_loss: 1.0755 - val_accuracy: 0.4167
Epoch 4/20
24/24 [==============================] - 17s 714ms/step - loss: 0.4893 - accuracy: 0.7938 - val_loss: 1.1747 - val_accuracy: 0.4167
Epoch 5/20
24/24 [==============================] - 17s 694ms/step - loss: 0.4179 - accuracy: 0.7787 - val_loss: 1.1785 - val_accuracy: 0.4167
Epoch 6/20
24/24 [==============================] - 17s 700ms/step - loss: 0.3340 - accuracy: 0.8520 - val_loss: 1.1908 - val_accuracy: 0.4167
Epoch 7/20
24/24 [==============================] - 16s 685ms/step - loss: 0.2775 - accuracy: 0.8508 - val_loss: 1.1280 - val_accuracy: 0.4167

In [26]:
# Evaluates the trained model on the Test dataset
model.evaluate(X_test, Y_test)

1/1 [==============================] - 1s 633ms/step - loss: 0.3358 - accuracy: 0.8750


[0.33578142523765564, 0.875]

Acknowledgement:

The dataset of Labeled images of Rice Leaves affected by three diseases used in this notebook is from
https://www.kaggle.com/tedisetiady/leaf-rice-disease-indonesia